# Setup


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import importlib
import tensorflow as tf
import numpy as np

# Modellib
import modellib as mlb
import modellib.io as io
import modellib.evaluate as eval

from tensorflow.keras.metrics import AUC

# Define Constants 

In [4]:
# Def constants and variables
NUM_NODES = 64
NUM_FEATURES = 21

# Load and Prepare Data

In [25]:
# Load data
train_bands, train_labels = io.read_from_hdf5("data/datasets/train_dataset_64_1600.h5")
val_bands, val_labels = io.read_from_hdf5("data/datasets/val_dataset_64_200.h5")
test_bands, test_labels = io.read_from_hdf5("data/datasets/test_dataset_64_200.h5")

# Printing shapes to verify
print("")
print(f"Train bands shape: {train_bands.shape}, Train labels shape: {train_labels.shape}")
print(f"Val bands shape: {val_bands.shape}, Validation labels shape: {val_labels.shape}")
print(f"Test bands shape: {test_bands.shape}, Test labels shape: {test_labels.shape}")

# Reshape
train_reshaped = np.squeeze(train_bands).transpose(0, 2, 1)
val_reshaped = np.squeeze(val_bands).transpose(0, 2, 1)
test_reshaped = np.squeeze(test_bands).transpose(0, 2, 1)

# Printing shapes to verify
print("")
print(f"Train Bands reshaped shape: {train_reshaped.shape}")
print(f"Val Bands reshaped shape: {val_reshaped.shape}")
print(f"Test Bands reshaped shape: {test_reshaped.shape}")

# Create initial adj & normalize
adj_matrix = mlb.gcn.create_initial_adj_matrix(NUM_NODES)
adj_matrix = adj_matrix / np.sum(adj_matrix, axis=1, keepdims=True)

# Repeat adj_matrix for each sample in train, val, and test sets
adj_matrix_train = np.repeat(adj_matrix[np.newaxis, :, :], train_reshaped.shape[0], axis=0)
adj_matrix_val = np.repeat(adj_matrix[np.newaxis, :, :], val_reshaped.shape[0], axis=0)
adj_matrix_test = np.repeat(adj_matrix[np.newaxis, :, :], test_reshaped.shape[0], axis=0)

# Printing shapes to verify
print("")
print(f"Init. Adjacency Matrix shape: {adj_matrix.shape}")
print(f"Train Adjacency Matrix shape: {adj_matrix_train.shape}")
print(f"Val Adjacency Matrix shape: {adj_matrix_val.shape}")
print(f"Test Adjacency Matrix shape: {adj_matrix_test.shape}")


Train bands shape: (1600, 21, 64, 1), Train labels shape: (1600, 64)
Val bands shape: (200, 21, 64, 1), Validation labels shape: (200, 64)
Test bands shape: (200, 21, 64, 1), Test labels shape: (200, 64)

Train Bands reshaped shape: (1600, 64, 21)
Val Bands reshaped shape: (200, 64, 21)
Test Bands reshaped shape: (200, 64, 21)

Init. Adjacency Matrix shape: (64, 64)
Train Adjacency Matrix shape: (1600, 64, 64)
Val Adjacency Matrix shape: (200, 64, 64)
Test Adjacency Matrix shape: (200, 64, 64)


# Define Model Parameters


In [6]:
# Learning Rate Schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

# Model Parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
class_weights = {0: 0.2, 1: 0.8}
weighted_binary_ce = lambda y_true, y_predictions: mlb.losses.weighted_binary_crossentropy(y_true, y_predictions, class_weights)
metrics = [AUC(multi_label=True)]

# Train Model

In [8]:
importlib.reload(mlb.gcn)

# Create Model 
model = mlb.gcn.create_gcn_model(NUM_NODES, NUM_FEATURES, use_attention=True)
model.compile(
    optimizer=optimizer,
    loss=weighted_binary_ce,
    metrics=metrics
)

# Fit Model
history = model.fit(
    [train_reshaped, adj_matrix_train],
    train_labels,
    epochs=100,
    batch_size=16,
    validation_data=([val_reshaped, adj_matrix_val], val_labels),
    class_weight=class_weights,
    callbacks=[early_stopping]
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 35s 180ms/step - auc: 0.5286 - loss: 1.5072 - val_auc: 0.6502 - val_loss: 0.8099
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 116ms/step - auc: 0.5998 - loss: 0.5447 - val_auc: 0.6587 - val_loss: 0.3670
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 121ms/step - auc: 0.6129 - loss: 0.1974 - val_auc: 0.6614 - val_loss: 0.2152
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - auc: 0.6196 - loss: 0.0803 - val_auc: 0.6615 - val_loss: 0.1666
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 138ms/step - auc: 0.6245 - loss: 0.0437 - val_auc: 0.6608 - val_loss: 0.1522
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 156ms/step - auc: 0.6163 - loss: 0.0331 - val_auc: 0.6594 - val_loss: 0.1484
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - auc: 0.6161 - loss: 0.0305 - val_auc: 0.6579 - val_loss: 0.1471
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 147ms/step - auc: 0.6160 - loss: 0.0297 - val_auc: 0.6575 - val_loss: 0.1469
Epoch 9/100
100/100 ━━━━

# Make Predictions

In [17]:
# Make predictions on the test set
y_pred_proba = model.predict([test_reshaped, adj_matrix_test])
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step


# Evaluate Model

In [18]:
# Evaluate model
gcn_metrics = eval.calculate_metrics(test_labels, y_pred)

Element-wise Accuracy: 0.883671875
Classification Report:
              precision    recall  f1-score   support

    no block       0.92      0.95      0.94     11516
       block       0.39      0.28      0.32      1284

    accuracy                           0.88     12800
   macro avg       0.66      0.61      0.63     12800
weighted avg       0.87      0.88      0.87     12800

Confusion Matrix:
[[10955   561]
 [  928   356]]
True Positives: 356
False Negatives: 928
True Negatives: 10955
False Positives: 561


# Save Predictions

In [26]:
# Save predicted blockstarts 
# np.save('data/blockstarts/gcn_predictions.npy', y_pred)